In [1]:
import os
import sys

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from src.models import ModelXtoC
from src.preprocessing.CUB import preprocessing_main
from src.utils import *
from config import PROJECT_ROOT
from src.training import run_epoch_x_to_c

from src.utils import find_class_imbalance
from config import CUB_CONFIG
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
N_TRIMMED_CONCEPTS = CUB_CONFIG['N_TRIMMED_CONCEPTS']

In [3]:
torch.mps.empty_cache()

In [4]:
torch.manual_seed(42)
concept_labels, train_loader, test_loader = preprocessing_main(class_concepts=False, verbose=True)

Found 11788 images.
Processing in 369 batches of size 32 (for progress reporting)...


Processing batches:  29%|██▉       | 108/369 [00:20<00:49,  5.25it/s]


KeyboardInterrupt: 

**Find device to run model on (CPU or GPU).**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available()
                    else "mps" if torch.backends.mps.is_available()
                    else "cpu")
print(f"Using device: {device}")

Using device: mps


### Loss


In [6]:
use_weighted_loss = True # Set to False for simple unweighted loss

if use_weighted_loss:
    concept_weights = find_class_imbalance(concept_labels)
    attr_criterion = [nn.BCEWithLogitsLoss(weight=torch.tensor([ratio], device=device, dtype=torch.float))
                    for ratio in concept_weights]
else:
    attr_criterion = [nn.BCEWithLogitsLoss() for _ in range(N_TRIMMED_CONCEPTS)]

In [7]:
def get_outputs_as_array(outputs, n_concepts):
    # Initialize an empty list to collect batches
    batch_results = []

    for i in range(len(outputs)):
        batch_size = outputs[i].shape[0]

        # Create a batch matrix with N_CONCEPTS number of columns
        batch_matrix = np.zeros((batch_size, n_concepts))

        for instance_idx in range(batch_size):
            # Extract, convert, and flatten data for the current concept
            instance_data = outputs[i][instance_idx].detach().cpu().numpy().flatten()
            batch_matrix[instance_idx, :] = instance_data

        # Add this consistently shaped batch matrix to our collection
        batch_results.append(batch_matrix)

    return np.vstack(batch_results)

# Load instance-based model

In [ ]:
model = torch.load(os.path.join(PROJECT_ROOT, 'models', 'CUB', 'instance_level_model.pth'), map_location=device, weights_only=False)

In [9]:
if train_loader:
    with torch.no_grad():
        shuffled_concept_labels = []
        shuffled_img_labels = []

        # Iterate through all batches
        for batch in train_loader:
            _, concept_labels, image_labels, _ = batch
            # Append batch labels to our list
            shuffled_concept_labels.append(concept_labels)
            shuffled_img_labels.append(image_labels)

        # Concatenate all batches into a single tensor
        shuffled_concept_labels = torch.cat(shuffled_concept_labels, dim=0)
        shuffled_img_labels = torch.cat(shuffled_img_labels, dim=0)

        test_loss, test_acc, outputs = run_epoch_x_to_c(
            model, train_loader, attr_criterion, optimizer=None, n_concepts=N_TRIMMED_CONCEPTS, device=device,
            return_outputs='sigmoid', verbose=True
        )

# print(f"Shuffled labels shape: {shuffled_img_labels.shape}")
np.save(os.path.join(PROJECT_ROOT, 'output', 'C_train_instance.npy'), shuffled_concept_labels)
np.save(os.path.join(PROJECT_ROOT, 'output', 'Y_train_instance.npy'), shuffled_img_labels)
print(f'Best Model Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

Best Model Summary   | Loss: 19.2846 | Acc: 81.834


In [10]:
output_array = get_outputs_as_array(outputs, N_TRIMMED_CONCEPTS)
print(f"Final shape: {output_array.shape}")

np.save(os.path.join(PROJECT_ROOT, 'output', 'C_hat_sigmoid_train_instance.npy'), output_array)

Final shape: (5994, 112)


# LOAD BEST MODEL FROM CBM PAPER

In [ ]:
best_model = os.path.join(PROJECT_ROOT, 'models', 'CUB', 'best_model_1.pth')
model = torch.load(best_model, map_location=device, weights_only=False)
print("Best model loaded.")

In [ ]:
if test_loader:
    with torch.no_grad():
        shuffled_concept_labels = []
        shuffled_img_labels = []

        # Iterate through all batches
        for batch in test_loader:
            _, concept_labels, image_labels, _ = batch
            # Append batch labels to our list
            shuffled_concept_labels.append(concept_labels)
            shuffled_img_labels.append(image_labels)

        # Concatenate all batches into a single tensor
        shuffled_concept_labels = torch.cat(shuffled_concept_labels, dim=0)
        shuffled_img_labels = torch.cat(shuffled_img_labels, dim=0)

        test_loss, test_acc, outputs = run_epoch_x_to_c(
            model, test_loader, attr_criterion, optimizer=None, n_concepts=N_TRIMMED_CONCEPTS, device=device,
            return_outputs='sigmoid', verbose=True
        )

# print(f"Shuffled labels shape: {shuffled_img_labels.shape}")
np.save(os.path.join(PROJECT_ROOT, 'output', 'C_test.npy'), shuffled_concept_labels)
np.save(os.path.join(PROJECT_ROOT, 'output', 'Y_test.npy'), shuffled_img_labels)
print(f'Best Model Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

In [ ]:
output_array = get_outputs_as_array(outputs, N_TRIMMED_CONCEPTS)
np.save(os.path.join(PROJECT_ROOT, 'output', 'C_hat_sigmoid_test.npy'), output_array)

In [ ]:
output_array[10]

In [ ]:
# best_model = os.path.join(PROJECT_ROOT, 'models', 'CUB', 'best_model_2.pth')
# model = torch.load(best_model, map_location=device, weights_only=False)
# print("Best model loaded.")

# if train_loader:
#     with torch.no_grad():
#         test_loss, test_acc = run_epoch_x_to_c(model, train_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device)

# print(f'Best Model Train Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

In [ ]:
# if test_loader:
#     with torch.no_grad():
#         test_loss, test_acc = run_epoch_x_to_c(model, test_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device)

# print(f'Best Model Test Summary    | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

In [ ]:
# best_model = os.path.join(PROJECT_ROOT, 'models', 'CUB', 'best_model_3.pth')
# model = torch.load(best_model, map_location=device, weights_only=False)
# print("Best model loaded.")

# if train_loader:
#     with torch.no_grad():
#         test_loss, test_acc = run_epoch_x_to_c(model, train_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device)

# print(f'Best Model Train Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

In [ ]:
# if test_loader:
#     with torch.no_grad():
#         test_loss, test_acc = run_epoch_x_to_c(model, test_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device)

# print(f'Best Model Test Summary    | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')